In [2]:
from commonfunctions import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
from skimage import feature
import numpy as np
import cv2
import math
from scipy import ndimage
import operator
%config IPCompleter.greedy=True



In [4]:
#cropping (kholy's part)
#work in img_rotated
#img.minval,maxval,kernel_size
#img_rotated=cv2.GaussianBlur(img_rotated,(11,11),0)

def find_page_contours(img):
    img2, contours, hierarchy = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours



def find_largest_contour(contours):
    contours_area_list = []
    for contour in contours:
        contours_area_list.append(cv2.contourArea(contour))
        
    index, value = max(enumerate(contours_area_list), key=operator.itemgetter(1))
    #print(value)
    return contours[index]


def findApprox(contours):
    ff = []
    approx_list = []
    for cnt in contours:
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.03 * perimeter, True)

        # Page has 4 corners and it is convex
        if (len(approx) == 4):
            #print(approx)
            maxArea = cv2.contourArea(approx)
            ff.append(maxArea)
            approx_list.append(approx) 

    index = findMax(ff)        
    return approx_list[index]     



def findMax(ff):
    index, value = max(enumerate(ff), key=operator.itemgetter(1))

    return index

     
def resize(img, height=800):
    """ Resize image to given height """
    if (img.shape[0] > height):
        ratio = height / img.shape[0]
        return cv2.resize(img, (int(ratio * img.shape[1]), height))    
    
    
def sortCornerPoints(points):
    print(points.shape)
    sorted_points = np.zeros_like(points)
    #add x to y
    #maximum sum -> top left
    #minimum sum -> top right
    sum_points = np.sum(points,axis=1)
    sorted_points[0] = points[np.argmin(sum_points)]
    sorted_points[2] = points[np.argmax(sum_points)]
    #maximum diff -> bottom left
    #minimum diff -> bottom right
    diff_points = np.diff(points,axis=1)
    print(points)
    sorted_points[1] = points[np.argmin(diff_points)]
    sorted_points[3] = points[np.argmax(diff_points)]
    return sorted_points
            
    
    
def transform_image(img,points):
    w = max(np.linalg.norm(points[0]-points[1]),np.linalg.norm(points[2]-points[3]))
    h = max(np.linalg.norm(points[0]-points[3]),np.linalg.norm(points[1]-points[2]))
    dest_img = np.array([[0,0],[w-1,0],[w-1,h-1],[0,h-1]])
    
    print(dest_img)
    print(points)
    
    dest_img = dest_img.astype(np.float32)
    points = points.astype(np.float32)

    Trans_Matrix = cv2.getPerspectiveTransform(points,dest_img)
    cropped_img = cv2.warpPerspective(img, Trans_Matrix, (int(w), int(h)))
    cv2.imwrite('output/gg.jpg',img)
    cv2.imwrite('output/cropped_edge.jpg',cropped_img)
    return


def crop_image(path):
    img_org = cv2.imread(path)
    
    img_before = resize(cv2.imread(path))
    img_gray = cv2.cvtColor(img_before, cv2.COLOR_BGR2GRAY)
    img_edges = cv2.bilateralFilter(img_gray, 9, 75, 75)
    img_edges = cv2.adaptiveThreshold(img_edges, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 4)
    img_edges = cv2.medianBlur(img_edges, 11)
    #img_edges = cv2.copyMakeBorder(img_edges, 5, 5, 5, 5, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    img_edges = cv2.Canny(img_edges,200, 250)
    img_edges = cv2.morphologyEx(img_edges, cv2.MORPH_CLOSE, np.ones((5, 11)))

    cv2.imwrite('output/2_edge.jpg',img_edges)

    pageContour = find_page_contours(img_edges)
    con = find_largest_contour(pageContour)
    #rect = cv2.minAreaRect(contour)
    #box = cv2.boxPoints(rect)
    #box = np.int0(box)
    #cv2.drawContours(img_before,[box],0,(0,0,255),2)
    #cv2.rectangle(img_before,(x,y),(x+w,y+h),(0,0,255),2)
    epsilon = 0.01*cv2.arcLength(con,True)
    approx = cv2.approxPolyDP(con,epsilon,True)
    ff= findApprox(pageContour)


    corner_points = sortCornerPoints(ff[:,0])*(img_org.shape[0]/800)
    cv2.drawContours(img_before,[ff],0,(0,0,255),3)
    cv2.imwrite('output/after_contour.jpg',img_before)



    transform_image(img_org,corner_points)
    
crop_image('output/2.jpg')





(4, 2)
[[482  50]
 [566 659]
 [152 732]
 [ 30 129]]
[[   0.            0.        ]
 [2274.90506656    0.        ]
 [2274.90506656 3050.48052407]
 [   0.         3050.48052407]]
[[ 148.8   639.84]
 [2390.72  248.  ]
 [2807.36 3268.64]
 [ 753.92 3630.72]]
